<a href="https://colab.research.google.com/github/korupolujayanth2004/DeepLearning/blob/main/House__Price_Prediction_using_Multi_layer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#dataset
#https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np

# Step 1: Load training and test data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Step 2: Select feature columns (numeric features, edit as needed)
features = [
    'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
    'TotalBsmtSF', 'GrLivArea', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea'
]
target = 'SalePrice'

# Step 3: Handle missing values in train and test features
X = train_df[features].fillna(train_df[features].mean())
y = train_df[target]
X_test_final = test_df[features].fillna(train_df[features].mean())

# Step 4: Split train data for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Scale numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test_final)

# Step 6: Build the MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=[X_train.shape[1]]),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Step 7: Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Step 8: Evaluate on validation set
loss, mae = model.evaluate(X_val, y_val)
print(f'Validation MAE: {mae:.2f}')

# Step 9: Make predictions for test set and display sample output
y_test_pred = model.predict(X_test).flatten()
print("Sample predicted prices on test set:", y_test_pred[:5])

# Step 10: Prepare submission file
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': y_test_pred
})
submission.to_csv('submission.csv', index=False)
print("Submission file saved as submission.csv")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 61ms/step - loss: 39082287104.0000 - mae: 182624.2969 - val_loss: 39647514624.0000 - val_mae: 178822.9375
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39927160832.0000 - mae: 182899.7188 - val_loss: 39595642880.0000 - val_mae: 178699.3594
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39009210368.0000 - mae: 181549.3281 - val_loss: 39348568064.0000 - val_mae: 178136.1250
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 38080139264.0000 - mae: 179884.7812 - val_loss: 38559969280.0000 - val_mae: 176366.3594
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36847984640.0000 - mae: 176085.1719 - val_loss: 36689657856.0000 - val_mae: 172137.4062
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35245596672.0000 - mae: 173303.8594 - val_loss: 33116848128.0000 - val_mae: 163764.0938
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31425001472.0000 - mae: 164278.9688 - val_loss: 2748162457

In [15]:
# --- Function for single input prediction ---

def predict_price(input_dict):
    input_df = pd.DataFrame([input_dict])
    for feat in features:
        if feat not in input_df.columns:
            input_df[feat] = train_df[feat].mean()
    input_df = input_df[features]
    input_df = input_df.fillna(train_df[features].mean())
    input_scaled = scaler.transform(input_df)
    pred_price = float(model.predict(input_scaled)[0])
    return pred_price

example_input = {
    'LotArea': 8450,
    'OverallQual': 7,
    'OverallCond': 5,
    'YearBuilt': 2003,
    'YearRemodAdd': 2003,
    'TotalBsmtSF': 856,
    'GrLivArea': 1710,
    'FullBath': 2,
    'HalfBath': 1,
    'BedroomAbvGr': 3,
    'TotRmsAbvGrd': 8,
    'GarageCars': 2,
    'GarageArea': 548
}

predicted_price = predict_price(example_input)
print(f"Predicted house price for example input: ${predicted_price:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
Predicted house price for example input: $213123.77


/tmp/ipython-input-3892468850.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_price = float(model.predict(input_scaled)[0])
